<a href="https://colab.research.google.com/github/YanPaivaAndrade/Minera-oDeDadosTrabalho3/blob/main/Trabalho_3_Yan_de_Paiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

texto em itálico# **Trabalho 3 - Aluno: Yan de Paiva Andrade Pinto**
Matricula: 201476038
Base de dados: **trabalho3_dados_29**

In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive




# **Tratamento dos dados**
  

*   Remoção de dados duplicados
*   Substituir '' por NaN
*   Substituir 'tdb' por NaN
*   Levantando numero de dados faltantes 

In [26]:
import pandas as pd
dadosTrabalho2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mineracao/trabalho2_dados_29.csv", dtype={'plataforma':'category'})
dadosTrabalho3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mineracao/trabalho3_dados_adicionais_29.csv", dtype={'plataforma':'category'})
dadosOriginais = dadosTrabalho2.append(dadosTrabalho3);
dados = dadosOriginais;
print (dadosOriginais.count())

dados = dados.drop_duplicates();
print(dados.count());
dados = dados.replace({'':float('nan')}).replace({'tbd':float('nan')})
dados_null_antes_agrupamento_nome = dados.isnull().sum()
print(dados_null_antes_agrupamento_nome)


nome                  3198
plataforma            3198
genero                3198
editora               3194
vendas                3198
lancamento            2462
avaliacao-criticos    2042
numero-criticos       2042
avaliacao-usuarios    2426
numero-usuarios       1938
fabricante            2450
dtype: int64
nome                  2428
plataforma            2428
genero                2428
editora               2425
vendas                2428
lancamento            1810
avaliacao-criticos    1480
numero-criticos       1480
avaliacao-usuarios    1784
numero-usuarios       1404
fabricante            1801
dtype: int64
nome                     0
plataforma               0
genero                   0
editora                  3
vendas                   0
lancamento             618
avaliacao-criticos     948
numero-criticos        948
avaliacao-usuarios    1024
numero-usuarios       1024
fabricante             627
dtype: int64


# **Transformação, Agrupamento de valores e Calculo da mediana por agrupamento.**
*   Atribuir tipagem numerica para as colunas: "avaliacao-criticos", "numero-criticos", "avaliacao-usuarios" e "numero-usuarios".
*   Agrupar dados por nome do jogo
*   Calcular a mediana dos determinados agrupamentos, a fim de tratar valores faltantes.

Com a definição do id da instancia como “nome”, torna-se indiferente o atributo "plataforma" para minha base de dados. Entretanto esse atributo gerou uma replicação de instancias, uma vez que o jogo poderia estar presente em múltiplas plataformas.

Também durante a analise da base de dados o atributo “fabricante” é ausente em cerca de 25% da base e atualmente não é possível inferir o mesmo com uma precisão satisfatória. Dito isto a tomada de decisão foi excluir este atributo.

Dito isto o objetivo inicial é completar os dados faltantes com maior realismo possível, pra isso agrupei os dados por nome a fim de calcular a mediana dos valores replicados. E substituir nas instancias com valores faltantes.



In [27]:
dados = dados.drop(columns=['plataforma'])
dados = dados.drop(columns=['fabricante'])
for column in dados.columns:
    if (dados[column].isnull().sum() > 0 and (column == "avaliacao-criticos"\
     or column == "numero-criticos" or column == "avaliacao-usuarios"\
      or column == "numero-usuarios")):
        dados[column] = dados[column].apply(pd.to_numeric)

tbl_avaliacao_criticos = dados[["nome","avaliacao-criticos"]]
median_avaliacao_criticos = tbl_avaliacao_criticos.groupby("nome").median()

tbl_numero_criticos = dados[["nome", "numero-criticos"]]
median_numero_criticos = tbl_numero_criticos.groupby("nome").median()

tbl_avaliacao_usuarios = dados[["nome", "avaliacao-usuarios"]]
median_avaliacao_usuarios = tbl_avaliacao_usuarios.groupby("nome").median()

tbl_numero_usuarios = dados[["nome", "numero-usuarios"]]
median_numero_usuarios = tbl_numero_usuarios.groupby("nome").median()


In [28]:
for e in median_avaliacao_criticos.index:
    tbl_avaliacao_criticos.loc[tbl_avaliacao_criticos["nome"] == e] =\
        tbl_avaliacao_criticos.loc[tbl_avaliacao_criticos["nome"] == e].\
            fillna(median_avaliacao_criticos["avaliacao-criticos"].get(e))
dados["avaliacao-criticos"] = tbl_avaliacao_criticos["avaliacao-criticos"]


In [29]:
for e in median_numero_criticos.index:
    tbl_numero_criticos.loc[tbl_numero_criticos["nome"] == e] =\
        tbl_numero_criticos.loc[tbl_numero_criticos["nome"] == e].\
            fillna(median_numero_criticos["numero-criticos"].get(e))
dados["numero-criticos"] = tbl_numero_criticos["numero-criticos"]

In [ ]:
for e in median_avaliacao_usuarios.index:
    tbl_avaliacao_usuarios.loc[tbl_avaliacao_usuarios["nome"] == e] =\
        tbl_avaliacao_usuarios.loc[tbl_avaliacao_usuarios["nome"] == e].\
            fillna(median_avaliacao_usuarios["avaliacao-usuarios"].get(e))
dados["avaliacao-usuarios"] = tbl_avaliacao_usuarios["avaliacao-usuarios"]

In [ ]:
for e in median_numero_usuarios.index:
    tbl_numero_usuarios.loc[tbl_numero_usuarios["nome"] == e] =\
        tbl_numero_usuarios.loc[tbl_numero_usuarios["nome"] == e].\
            fillna(median_numero_usuarios["numero-usuarios"].get(e))
dados["numero-usuarios"] = tbl_numero_usuarios["numero-usuarios"]

In [32]:
print(dados_null_antes_agrupamento_nome)
print(dados.isnull().sum())

nome                     0
plataforma               0
genero                   0
editora                  3
vendas                   0
lancamento             618
avaliacao-criticos     948
numero-criticos        948
avaliacao-usuarios    1024
numero-usuarios       1024
fabricante             627
dtype: int64
nome                    0
genero                  0
editora                 3
vendas                  0
lancamento            618
avaliacao-criticos    829
numero-criticos       829
avaliacao-usuarios    873
numero-usuarios       873
dtype: int64


# **Tratamento final de valores faltantes.**

Para tratar os valores ‘NaN’ ainda presentes no DF, minha decisão foi agrupar o DF pelo atributo “Gênero”, calcular a mediana e substituir os valores faltantes daquele determinado gênero por sua respectiva mediana.
O intuito deste tratamento é definir um comportamento para a comunidade “gamer’’ daquele determinado gênero.


In [33]:
tbl_avaliacao_criticos_genero = dados[["genero","avaliacao-criticos"]]
median_avaliacao_criticos_genero = tbl_avaliacao_criticos_genero.groupby("genero").median()

tbl_numero_criticos_genero = dados[["genero", "numero-criticos"]]
median_numero_criticos_genero = tbl_numero_criticos_genero.groupby("genero").median()

tbl_avaliacao_usuarios_genero = dados[["genero", "avaliacao-usuarios"]]
median_avaliacao_usuarios_genero = tbl_avaliacao_usuarios_genero.groupby("genero").median()

tbl_numero_usuarios_genero = dados[["genero", "numero-usuarios"]]
median_numero_usuarios_genero = tbl_numero_usuarios_genero.groupby("genero").median()


In [ ]:
for e in median_avaliacao_criticos_genero.index:
    tbl_avaliacao_criticos_genero.loc[tbl_avaliacao_criticos_genero["genero"] == e] =\
        tbl_avaliacao_criticos_genero.loc[tbl_avaliacao_criticos_genero["genero"] == e].\
            fillna(median_avaliacao_criticos_genero["avaliacao-criticos"].get(e))
dados["avaliacao-criticos"] = tbl_avaliacao_criticos_genero["avaliacao-criticos"]

In [ ]:
for e in median_numero_criticos_genero.index:
    tbl_numero_criticos_genero.loc[tbl_numero_criticos_genero["genero"] == e] =\
        tbl_numero_criticos_genero.loc[tbl_numero_criticos_genero["genero"] == e].\
            fillna(median_numero_criticos_genero["numero-criticos"].get(e))
dados["numero-criticos"] = tbl_numero_criticos_genero["numero-criticos"]

In [ ]:
for e in median_avaliacao_usuarios_genero.index:
    tbl_avaliacao_usuarios_genero.loc[tbl_avaliacao_usuarios_genero["genero"] == e] =\
        tbl_avaliacao_usuarios_genero.loc[tbl_avaliacao_usuarios_genero["genero"] == e].\
            fillna(median_avaliacao_usuarios_genero["avaliacao-usuarios"].get(e))
dados["avaliacao-usuarios"] = tbl_avaliacao_usuarios_genero["avaliacao-usuarios"]

In [ ]:
for e in median_numero_usuarios_genero.index:
    tbl_numero_usuarios_genero.loc[tbl_numero_usuarios_genero["genero"] == e] =\
        tbl_numero_usuarios_genero.loc[tbl_numero_usuarios_genero["genero"] == e].\
            fillna(median_numero_usuarios_genero["numero-usuarios"].get(e))
dados["numero-usuarios"] = tbl_numero_usuarios_genero["numero-usuarios"]

In [39]:
print(dados.isnull().sum())

nome                    0
genero                  0
editora                 3
vendas                  0
lancamento            618
avaliacao-criticos      0
numero-criticos         0
avaliacao-usuarios      0
numero-usuarios         0
dtype: int64
